<a href="https://colab.research.google.com/github/hackerpranavpandey/SOI_Mission_Mars_Terrain/blob/main/Test_ResNet152.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
import os
path='/content/drive/MyDrive/Mission_Mars'
os.listdir(path)

['train.csv',
 'train_dataset',
 'test_dataset',
 'pred_resnet152.csv',
 'MissionMarsTerrain_submission.csv']

In [60]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
images=[]
image_filenames=[]
def image_process(images_path):
    for image_path in os.listdir(images_path):
        image_name = os.path.join(images_path, image_path)
        image_array = cv2.imread(image_name, cv2.IMREAD_COLOR)
        image_array = cv2.resize(image_array, (224, 224))
        image_array = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)
        images.append(image_array)
        image_filenames.append(image_path)
image_process('/content/drive/MyDrive/Mission_Mars/test_dataset')

In [61]:
import torch
from torch.utils.data import Dataset
from PIL import Image
class CustomDataset(Dataset):
    def __init__(self,images,transform=None):
        self.images = images
        self.transform = transform
    def __len__(self):
        return len(self.images)
    def __getitem__(self, idx):
        image = self.images[idx]
        image = Image.fromarray(image)
        if self.transform:
            image = self.transform(image)
        return image

In [62]:
from torchvision import transforms
data_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                     ])

In [63]:
from torch.utils.data import DataLoader
test_dataset = CustomDataset(images, transform=data_transforms)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

In [64]:
import torch
import torchvision.models as models
from torch import nn
num_classes = 8
model = models.resnet152(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model_path = '/content/drive/MyDrive/model.pth'
model.load_state_dict(torch.load(model_path))

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

In [65]:
def get_predictions(model, test_loader):
    model.eval()
    all_outputs = []
    with torch.no_grad():
        for inputs in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_outputs.extend(predicted.cpu().numpy())
    return all_outputs

In [66]:
label_mapping = {
    0: 'bright dune',
    1: 'crater',
    2: 'dark dune',
    3: 'impact ejecta',
    4: 'other',
    5: 'slope streak',
    6: 'spider',
    7: 'swiss cheese'
}
pred_labels = get_predictions(model, test_loader)
decoded_labels=[label_mapping[label] for label in pred_labels]
import pandas as pd
results = pd.DataFrame({
    'File Name': image_filenames,
    'Class': decoded_labels
})
results.to_csv('/content/drive/MyDrive/Mission_Mars/MissionMarsTerrain_submission.csv', index=False)
print("Predictions saved to CSV file.")

Predictions saved to CSV file.


In [70]:
import pandas as pd
file_path = '/content/drive/MyDrive/Mission_Mars/MissionMarsTerrain_submission.csv'
results = pd.read_csv(file_path)
final_class=[]
for i in range(2000):
  final_class.append(0)

In [71]:
index=results.iloc[:,0]

In [72]:
for i in range(0,len(index)):
  t=index[i].split('.')[0]
  if len(t)>4:
    continue
  t=int(t)
  final_class[t-1]=results.iloc[i,1]

In [73]:
len(final_class)

2000

In [76]:
for i in range(0,20):
  print(final_class[i])

slope streak
other
crater
other
other
other
bright dune
other
other
crater
crater
crater
crater
crater
bright dune
other
crater
bright dune
other
other


In [77]:
image_name=[]
for i in range(0,2000):
  image_name.append(str(i+1)+'.jpg')

In [78]:
predicted_csv = pd.DataFrame({
    'File Name': image_name,
    'Class': final_class
})
predicted_csv.to_csv('/content/drive/MyDrive/Mission_Mars/MissionMarsTerrain_submission.csv', index=False)
print("Predictions saved to CSV file.")

Predictions saved to CSV file.
